In [ ]:
import pandas as pd

In [ ]:
from notebook.auth import passwd; passwd()

In [ ]:
aggregated_clear = pd.read_csv('../data/marks_csv/aggregated_clear.csv', sep=',')
aggregated_clear.head()

In [ ]:
aggregated_normalize = pd.read_csv('../data/marks_csv/aggregated_normalize.csv', sep=',')
aggregated_normalize.head()

In [ ]:
# aggregated_normalize

In [ ]:
with open('aggregated_clear_fasttext.txt', 'w', encoding='utf-8') as fp:
    for _, row in aggregated_clear.iterrows():
        new_row = '__label__' + row['output']+ ' ' + row['input'] + '\n'
        fp.write(new_row)

In [ ]:
with open('aggregated_normalize_fasttext.txt', 'w', encoding='utf-8') as fp:
    for _, row in aggregated_normalize.iterrows():
        new_row = '__label__' + row['output']+ ' ' + row['input_normal_form'] + '\n'
        fp.write(new_row)

# Train model

In [ ]:
import fasttext

In [ ]:
for wordNgrams in (1, 2):
    for lr in (0.1, 0.5, 1.0):
        for epoch in (5, 10, 20, 30, 40):
            model = fasttext.train_supervised(input="./data/aggregated_clear_fasttext_train.txt",
                                              epoch=epoch, lr=lr, wordNgrams=wordNgrams)
            print(f"wordNgrams: {wordNgrams}, lr: {lr}, epoch: {epoch}",
                  model.test("./data/aggregated_clear_fasttext_test.txt"))
            # print(model.test("./data/aggregated_clear_fasttext_train.txt"))
            print()
        print("lr")
    print("wordNgrams")

wordNgrams: 1, lr: 1.0, epoch: 5 (1150, 0.5756521739130435, 0.5756521739130435)

In [ ]:
for wordNgrams in (1, 2):
    for lr in (0.1, 0.5, 1.0, 2.0):
        for epoch in (5, 10, 20, 30, 40):
            model_normalize = fasttext.train_supervised(input="./data/aggregated_normalize_fasttext_train.txt",
                                                        epoch=epoch, lr=lr, wordNgrams=wordNgrams)
            print(f"wordNgrams: {wordNgrams}, lr: {lr}, epoch: {epoch}",
                  model_normalize.test("./data/aggregated_normalize_fasttext_test.txt"))
        print("lr")
    print("wordNgrams")

wordNgrams: 1, lr: 0.5, epoch: 5 (1150, 0.5791304347826087, 0.5791304347826087)

# Download models

In [ ]:
import fasttext.util
import pandas as pd

In [ ]:
fasttext.util.download_model('ru', if_exists='ignore')

In [ ]:
ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:
aggregated_clear = pd.read_csv('./data/aggregated_clear.csv')
aggregated_clear.head()
res = []
for _, row in aggregated_clear.iterrows():
    res.append(ft.get_sentence_vector(row['input']))

df_res = pd.DataFrame(res)
df_res['output'] = aggregated_clear['output']

In [ ]:
df_res.to_csv('./data/aggregated_clear_vectors.csv', index=None)

In [ ]:
import fasttext.util
import pandas as pd

fasttext.util.download_model('ru', if_exists='ignore')

ft = fasttext.load_model('cc.ru.300.bin')

aggregated_clear = pd.read_csv('./data/aggregated_normalize.csv')
aggregated_clear.head()
res = []
for _, row in aggregated_clear.iterrows():
    res.append(ft.get_sentence_vector(row['input_normal_form']))

df_res = pd.DataFrame(res)
df_res['output'] = aggregated_clear['output']

df_res.to_csv('./data/aggregated_normalize_vectors.csv', index=None)